## I plan to build recommendation system on google gcloud. I want to make sure all codes are good before gcloud.
### Thus, I set up spark in google colab with TPU.
### the data and codes are from:
#### https://github.com/GoogleCloudPlatform/spark-recommendation-engine
### Part 2: use the best parameter in ALS model for prediction

In [1]:
!ls

 accommodations.csv	     implicit_ratings.csv
 drive			     ratings.csv
'implicit_ratings (1).csv'   recommender_model01_0116_selected.csv
'implicit_ratings (2).csv'  'recommender_model_0116 (1).csv'
'implicit_ratings (3).csv'   recommender_model_0116.csv
'implicit_ratings (4).csv'   sample_data


In [33]:
from __future__ import print_function

import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType

conf = SparkConf().setAppName("app_collaborative")
sc = SparkContext(conf=conf)
sc.setCheckpointDir('checkpoint/')
sqlContext = SQLContext(sc)

USER_ID = 0


ValueError: ignored

In [0]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import Bucketizer, MinMaxScaler, VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Transformer
from pyspark.sql import Window

from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
# dfAccos = sqlContext.read.jdbc(url=jdbcUrl, table=TABLE_ITEMS)
# dfRates = sqlContext.read.jdbc(url=jdbcUrl, table=TABLE_RATINGS)

In [0]:
#dfAccos = spark.read.csv('accommodations.csv', header=True, inferSchema=True).cache()
dfAccos = spark.read.csv('accommodations.csv')

In [23]:
pd.DataFrame(dfAccos.take(5), columns=dfAccos.columns)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,1,Comfy Quiet Chalet,Vancouver,50,3,3.1,cottage
1,2,Cozy Calm Hut,London,65,2,4.1,cottage
2,3,Agreable Calm Place,London,65,4,4.8,house
3,4,Colossal Quiet Chateau,Paris,3400,16,2.7,castle
4,5,Homy Quiet Shack,Paris,50,1,1.1,cottage


In [0]:
dfAccos = dfAccos.withColumnRenamed('_c0', 'id')\
.withColumnRenamed('_c1', 'title')\
.withColumnRenamed('_c2', 'location')\
.withColumnRenamed('_c3', 'price')\
.withColumnRenamed('_c4', 'rooms')\
.withColumnRenamed('_c5', 'rating')\
.withColumnRenamed('_c6', 'type')\


In [25]:
pd.DataFrame(dfAccos.take(5), columns=dfAccos.columns)

,id,title,location,price,rooms,rating,type
0,1,Comfy Quiet Chalet,Vancouver,50,3,3.1,cottage
1,2,Cozy Calm Hut,London,65,2,4.1,cottage
2,3,Agreable Calm Place,London,65,4,4.8,house
3,4,Colossal Quiet Chateau,Paris,3400,16,2.7,castle
4,5,Homy Quiet Shack,Paris,50,1,1.1,cottage


In [0]:
dfRates = spark.read.csv('ratings.csv')

In [16]:
pd.DataFrame(dfRates.take(5), columns=dfRates.columns)

,_c0,_c1,_c2
0,10,1,1
1,18,1,2
2,13,1,1
3,7,2,2
4,4,2,2


In [0]:
dfRates = dfRates.withColumnRenamed('_c0', 'userId')\
.withColumnRenamed('_c1', 'accoId')\
.withColumnRenamed('_c2', 'rating')\

In [27]:
pd.DataFrame(dfRates.take(5), columns=dfRates.columns)

,userId,accoId,rating
0,10,1,1
1,18,1,2
2,13,1,1
3,7,2,2
4,4,2,2


In [0]:
USER_ID = 0

In [29]:
# Get all the ratings rows of our user
dfUserRatings  = dfRates.filter(dfRates.userId == USER_ID).rdd.map(lambda r: r.accoId).collect()
print(dfUserRatings)

['3', '6', '12', '16', '22', '28', '30', '33', '38']


In [0]:
# Returns only the accommodations that have not been rated by our user
rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
pairsPotential = rddPotential.map(lambda x: (USER_ID, x[0]))

In [0]:
#[START split_sets]
rddTraining, rddValidating, rddTesting = dfRates.rdd.randomSplit([6,2,2])

In [0]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [0]:
#[START predict]
# Build our model with the best found values
# Rating, Rank, Iteration, Regulation
model = ALS.train(rddTraining, 15, 20, 0.1)


In [0]:
# Calculate all predictions
predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))

In [37]:
# Take the top 5 ones
topPredictions = predictions.takeOrdered(5, key=lambda x: -x[2])
print(topPredictions)

[('0', '49', 4.896520089114775), ('0', '76', 4.754601059938514), ('0', '66', 4.707847057758075), ('0', '75', 4.698511918681345), ('0', '72', 4.254802137130806)]


In [0]:
schema = StructType([StructField("userId", StringType(), True), StructField("accoId", StringType(), True), StructField("prediction", FloatType(), True)])

In [0]:
dfToSave = sqlContext.createDataFrame(topPredictions, schema)

In [40]:
dfToSave

DataFrame[userId: string, accoId: string, prediction: float]

In [41]:
dfToSave.show()

+------+------+----------+
|userId|accoId|prediction|
+------+------+----------+
|     0|    49|   4.89652|
|     0|    76|  4.754601|
|     0|    66|  4.707847|
|     0|    75|  4.698512|
|     0|    72|  4.254802|
+------+------+----------+

